# Approach to solve it

- **Read the csv file that contains all the meta informtion needed for scraping**
- **Iterate through each row in csv and get the ASIN and country code**
- **Build the url dynamically and call it using requests library**
- **Based on the status code execute the condition, if the status code is 404 then the URL is not found else parse the html response using beautiful soup**
- **Extract 4 key information - product price, product image url, product details and product title**
- **Once 100 product information scraped, generate a json file and exit from program with a message**

In [3]:
# Import necessary libraries
import urllib3
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import sys

In [4]:
# Read csv that contains the asin and contry information
df = pd.read_csv("Amazon Scraping - Sheet1.csv")
df.head()

,Unnamed: 0,id,Asin,country
0,0,1,1015,de
1,1,2424796,1015,fr
2,2,2,000004458X,de
3,3,2424797,000004458X,fr
4,4,3,1002198,de


In [5]:
# Function to scrape the Amazon pdoduct details by passing the asin and country code
def get_product_info(country, asin, id_):
    url = "https://www.amazon.{0}/dp/{1}".format(country, asin)
    try:
        page = requests.get(url, headers = headers)
        if page.status_code == 404:
            print("{} not available".format(url))
        else:
            soup = BeautifulSoup(page.text, 'html.parser')
            p_title = soup.find('span', attrs={"id":'productTitle'}).text.strip()
            p_price = soup.find('span', attrs={"class":'a-offscreen'}).text.strip()
            p_img_url = eval(soup.findAll('script', attrs={"type":'a-state'})[2].text)["landingImageUrl"].strip()
            p_details = soup.find('div', attrs={"id":'productAlert_feature_div'}).text.strip()
            if not p_details:
                p_details = soup.find('div', attrs={"id":'featurebullets_feature_div'}).text.strip()
            p_dict = {"Product Title": p_title, "Product Image URL":p_img_url, "Price of the Product": p_price, "Product Details": p_details}
            output.append(p_dict)
    except requests.exceptions.HTTPError as e:
        #print("http error")
        pass
    except requests.exceptions.Timeout as e:
        pass
        #print("Timeout error")
    except requests.exceptions.RequestException as e:
        pass
        #print("Request rejected error")
    except requests.exceptions.ConnectionError as e:
        pass
        #print("Connection error")
    except Exception as e:
        print("Oops! something went wrong.")
        pass
    
    try:
        if (len(output) == 100) or (id_ == 99):
            print("entered.....................................")
            with open("amazon_product_details.json", "w", encoding="utf-8") as fp:
                fp.write(str(output))
            print("Total time to get 100 product details (including skipped one's) in minutes: ", (time.time() - start_time)/60)
            sys.exit("Completed scraping of 100 products")
    except Exception as e:
        print(e)

In [32]:
# Iterate through each country code and ASIN no
headers = {"Accept-Language": "*",
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}
output = []
start_time = time.time()
for id_, row in df[:100].iterrows():
    get_product_info(row["country"], row["Asin"], id_)
    print("Number of rows completed: ", id_)

https://www.amazon.de/dp/1015 not available
Number of rows completed:  0
https://www.amazon.fr/dp/1015 not available
Number of rows completed:  1
Number of rows completed:  2
https://www.amazon.fr/dp/000004458X not available
Number of rows completed:  3
https://www.amazon.de/dp/1002198 not available
Number of rows completed:  4
https://www.amazon.fr/dp/1002198 not available
Number of rows completed:  5
https://www.amazon.fr/dp/1002791 not available
Number of rows completed:  6
https://www.amazon.it/dp/1002791 not available
Number of rows completed:  7
https://www.amazon.de/dp/1002864 not available
Number of rows completed:  8
https://www.amazon.fr/dp/1002864 not available
Number of rows completed:  9
https://www.amazon.de/dp/1003704 not available
Number of rows completed:  10
https://www.amazon.fr/dp/1003704 not available
Number of rows completed:  11
https://www.amazon.de/dp/1003763 not available
Number of rows completed:  12
https://www.amazon.fr/dp/1003763 not available
Number of ro

SystemExit: Completed scraping of 100 products

C:\Users\premvardhan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


- **This can be done for n number of URL**